In [43]:
import gensim
from gensim import corpora
import codecs
import MySQLdb as mdb
import pandas as pd
import re
import jieba
from pandas import Series,DataFrame
import numpy as np

### 准备数据

In [4]:
def mysql_con():
    try:
        db = mdb.connect('mysql-slave.communion.net.cn', 'onlyquery', 'jyhc@onlyquery', 'hainan_bigdata', charset='utf8')
        cursor = db.cursor()
    except:
        time.sleep(10)
        db = mdb.connect('mysql-slave.communion.net.cn', 'onlyquery', 'jyhc@onlyquery', 'hainan_bigdata', charset='utf8')
        cursor = db.cursor()
    return cursor,db

In [19]:
stop_words = set()
with codecs.open('stop_words2.txt','r','utf8') as f:
    for word in f.readlines():
        stop_words.add(word.strip())

In [159]:
#连接数据库并读取数据
cursor,db = mysql_con()
article = pd.read_sql('select tbl_book.title as b_title,tbl_book.author,tbl_article.content,tbl_article.title as c_title from tbl_book,tbl_article where tbl_book.id=tbl_article.bookId limit 5000',con=db)

In [160]:
def seg_words(words,stop_words):
    return [word for word in jieba.cut(words) if len(word)>1 and word not in stop_words]
#通过正则去掉数字和字母
article_content = article.content.apply(lambda x:re.sub(u'[\s\w+-.?!]','',x,flags=re.S))

In [161]:
#对数据库内的美文按照要求进行分词
temp = article_content.apply(lambda x:seg_words(x,stop_words)) 
texts = temp + article.b_title.apply(lambda x:seg_words(x,stop_words)) + article.author.apply(lambda x:seg_words(x,stop_words))+article.c_title.apply(lambda x:seg_words(x,stop_words))

In [162]:
#把text保存到文本文件，以方便后面的流式处理
corpora_texts = texts.apply(lambda x:u' '.join(x))
with codecs.open('corpora.texts.txt','w',encoding='utf8') as f:
    for doc in corpora_texts:
        f.write(doc+'\n')

### 打开建立好的模型及字典

In [53]:
models = gensim.models.LdaModel.load('wiki/meiwen_lda.pkl', mmap='r')  #注意，模型保存的时候一定是corpus和id2word一起保存。
dictionary = gensim.corpora.Dictionary.load_from_text('wiki/wiki_en_output_words.txt')

In [54]:
#把模型的100个主题和前30个关键词写入wiki_models.txt文本以便查看
with codecs.open('wiki_models.txt','w',encoding='utf8') as f:
    for i in models.show_topics(100,30):
        f.write(i[1]+'\n')
        
#从文本中读取这些主题进行查看
wiki_model = []
with codecs.open('wiki_models.txt','r',encoding='utf8') as f:
    for i in f.readlines():
        wiki_model.append(i)

### 对之前的语料进行测试

In [163]:
#对文件进行流式处理，这样就不用把文件全部读取到内存中了。并且每次只处理一条语句，每次根据算法循环到yield返回
#如果该类没有next()方法，就不能通过next(Mycorpus)进行迭代
class Mycorpus(object):
    def __iter__(self):
        for line in open('corpora.texts.txt'):
            yield dictionary.doc2bow(line.split())
corpus = Mycorpus()

In [164]:
thetas = np.zeros((len(texts), 100))  #每一行代表一个语料，每一列代表一个主题
for i,c in enumerate(corpus):
    if i%100 == 0:
        print 'now the total doc is:%s'%i
    for ti, v in models[c]:
        thetas[i,ti] += v

now the total doc is:0
now the total doc is:100
now the total doc is:200
now the total doc is:300
now the total doc is:400
now the total doc is:500
now the total doc is:600
now the total doc is:700
now the total doc is:800
now the total doc is:900
now the total doc is:1000
now the total doc is:1100
now the total doc is:1200
now the total doc is:1300
now the total doc is:1400
now the total doc is:1500
now the total doc is:1600
now the total doc is:1700
now the total doc is:1800
now the total doc is:1900
now the total doc is:2000
now the total doc is:2100
now the total doc is:2200
now the total doc is:2300
now the total doc is:2400
now the total doc is:2500
now the total doc is:2600
now the total doc is:2700
now the total doc is:2800
now the total doc is:2900
now the total doc is:3000
now the total doc is:3100
now the total doc is:3200
now the total doc is:3300
now the total doc is:3400
now the total doc is:3500
now the total doc is:3600
now the total doc is:3700
now the total doc is:380

In [370]:
thetas.shape
#保存theta矩阵
np.save('topics_hdp.npy', thetas)

In [375]:
#load获取theta
theta = np.load('topics_hdp.npy')

In [167]:
#计算矩阵行之间的距离并转化为矩阵
from scipy.spatial import distance

In [174]:
#先把矩阵转化为每个观测值和其余所有值的距离(默认是阿基里的距离)，然后在转化为矩阵
#把矩阵对角线（自己和自己）的距离设置为最大（之前肯定是最小的）
distances = distance.squareform(distance.pdist(thetas))
large = distances.max() + 1
for i in range(len(distances)):
    distances[i, i] = large

### 通过id查询最相似的美文

In [300]:
query_id = 16

##### 根据已有的美文查询

In [410]:
#查询
query_id = 230
print article.iloc[query_id]
print article.content[query_id]

b_title                                                   嚎叫
author                                           [美] 艾伦·金斯伯格
content         “我们是最工业的城市，我们有最摇滚的战士：\n    最工业化的号角吹响，在大洋彼...
c_title                                         不是每一个男人都如他坦荡
Name: 230, dtype: object
     “我们是最工业的城市，我们有最摇滚的战士：
    最工业化的号角吹响，在大洋彼岸的美利坚；在日耳曼；在菊花国度，在中国。人类的贪婪绑架了机械和自己的灵魂。这是每个人和人类的角力，是天空和火焰；大地同海洋的悬浊—痛苦、和声嘶力竭。


In [406]:
#主题
for i,v in models[dictionary.doc2bow(texts[query_id])][:5]:
    print "relevancy is: ",v
    print wiki_model[i]

relevancy is:  0.625334305774
0.009*社会 + 0.008*国家 + 0.008*政府 + 0.007*政治 + 0.006*经济 + 0.006*运动 + 0.006*组织 + 0.005*法律 + 0.005*认为 + 0.004*问题 + 0.004*美国 + 0.004*发展 + 0.004*自由 + 0.004*支持 + 0.004*反对 + 0.004*包括 + 0.004*年代 + 0.004*提出 + 0.004*政策 + 0.004*宪法 + 0.003*法案 + 0.003*影响 + 0.003*文化 + 0.003*改革 + 0.003*制度 + 0.003*成为 + 0.003*活动 + 0.003*中国 + 0.003*进行 + 0.003*历史

relevancy is:  0.0116747691697
0.032*世纪 + 0.021*帝国 + 0.021*王国 + 0.016*时期 + 0.013*王朝 + 0.011*公元前 + 0.009*现今 + 0.009*公元 + 0.008*历史 + 0.008*统治 + 0.008*琉球 + 0.008*时代 + 0.008*部落 + 0.005*波斯 + 0.005*民族 + 0.005*文化 + 0.005*地区 + 0.005*建立 + 0.005*希腊 + 0.004*国王 + 0.004*征服 + 0.004*菲律宾 + 0.004*文明 + 0.004*记载 + 0.004*称为 + 0.004*拜占庭 + 0.003*成为 + 0.003*神庙 + 0.003*西亚 + 0.003*古代

relevancy is:  0.176030494418
0.012*认为 + 0.005*问题 + 0.004*进行 + 0.004*出现 + 0.004*影响 + 0.004*之后 + 0.004*已经 + 0.004*存在 + 0.003*表示 + 0.003*内容 + 0.003*工作 + 0.003*一种 + 0.003*包括 + 0.003*需要 + 0.003*行为 + 0.003*选择 + 0.003*时间 + 0.003*指出 + 0.003*方式 + 0.003*评价 + 0.003*无法 + 0.002*表现 + 0.002*

In [407]:
sim_id = distances[query_id].argmin()
for sim_id in distances[query_id].argsort()[:10]:
    print 'index_id:',sim_id,'  ','sim value is:',distances[query_id][sim_id]
    print article[['b_title','author','c_title']].iloc[sim_id]
    print 'content: ', article.content[sim_id][:100]
    print "*"*100

index_id: 693    sim value is: 0.0869786099252
b_title                    政治经济学的国民体系
author                      弗里德里希.李斯特
c_title    看来贸易保护措施并不一定都是坏事,不能盲目崇拜...
Name: 693, dtype: object
content:  1》一国的在自己的工业、农业尚不具备条件和外国成熟企业竞争时，限制性的保护措施不必可少。等待国内的工业足够与外国资本竞争时再实行自由贸易才是历史给我的经验教训。

2》看来贸易保护措施并不一定都是坏事
****************************************************************************************************
index_id: 690    sim value is: 0.102612087239
b_title    政治经济学的国民体系
author      弗里德里希.李斯特
c_title     不单是关税保护制度
Name: 690, dtype: object
content:  讲到近代德国的统一，在经济方面往往必谈1834年由普鲁士领导形成的德意志关税同盟，这常被看作是德国政治统一在经济上的前提；而当提及德意志关税同盟之时，对于弗里德里希•李斯特又是不能避而不谈的。李斯特被
****************************************************************************************************
index_id: 3153    sim value is: 0.107884262972
b_title    公共领域的结构转型
author       （德）哈贝马斯
c_title      失落的公共领域
Name: 3153, dtype: object
content:  尤根·哈贝马斯（Jurgen Habermas），1929生于德国杜塞尔多夫。1949年起，先后在哥廷根、苏黎世、波恩上大学，学习哲学、历史、心理学、文学等，1954年博士学位。1956年，来到法兰克
******

In [309]:
distances[query_id]

array([ 0.77794578,  0.74031204,  0.63255605, ...,  0.61311509,
        0.63483918,  0.66312963])

#### 新增文章实时查询

In [413]:
query_text = u'瑰丽多姿的少数民族'
temp = [dictionary.doc2bow([word]) for word in jieba.cut(query_text,cut_all=True) if len(word)>1 and word not in stop_words]

In [414]:
for i in map(lambda x:models[x],temp):
    print wiki_model[i[0][0]]

0.131*香港 + 0.007*硬币 + 0.005*铸币 + 0.005*主席 + 0.004*举行 + 0.004*日报 + 0.003*包括 + 0.003*锦标 + 0.003*第三日 + 0.003*美分 + 0.003*市民 + 0.003*参与 + 0.003*人士 + 0.003*金币 + 0.003*华侨 + 0.003*活动 + 0.002*发行 + 0.002*网上 + 0.002*总会 + 0.002*内地 + 0.002*华人 + 0.002*周年 + 0.002*期间 + 0.002*纪念 + 0.002*英国 + 0.002*传媒 + 0.002*澳门 + 0.002*正面 + 0.002*梁振英 + 0.002*慈善

0.015*中国 + 0.010*记载 + 0.005*时期 + 0.005*孔子 + 0.005*古代 + 0.004*朝鲜 + 0.004*历史 + 0.004*出自 + 0.004*学者 + 0.004*姓氏 + 0.004*春秋 + 0.003*文献 + 0.003*高丽 + 0.003*见于 + 0.003*认为 + 0.003*史记 + 0.002*唐代 + 0.002*内容 + 0.002*历代 + 0.002*史料 + 0.002*人物 + 0.002*编纂 + 0.002*天下 + 0.002*楚国 + 0.002*出土 + 0.002*三国 + 0.002*宋代 + 0.002*新罗 + 0.002*中原 + 0.002*公子

0.027*语言 + 0.024*英语 + 0.017*使用 + 0.011*方言 + 0.010*字母 + 0.009*法语 + 0.007*发音 + 0.006*动词 + 0.006*范例 + 0.006*日语 + 0.006*名词 + 0.006*汉字 + 0.006*元音 + 0.006*表示 + 0.006*德语 + 0.005*汉语 + 0.005*意思 + 0.005*中文 + 0.004*文字 + 0.004*一种 + 0.004*西班牙语 + 0.004*形式 + 0.004*声母 + 0.004*变化 + 0.003*语音 + 0.003*通常 + 0.003*书写 + 0.003*人称 + 0.003*辅音 + 0.003*词汇

0.016*山西省

### 知识点  对繁体中文转化为简体

In [ ]:
from opencc import OpenCC 

In [ ]:
# opencc真的很强大，api也很简单，对整个wiki中文进行的转化也只用了半小时
converted = openCC.convert(str_line)